In [89]:
import tensorflow as tf
import pickle
from util import *

In [91]:
tf.reset_default_graph() 

# Add ops to save and restore all the variables.
feature_size = 10001
hidden_size = 100
target_size = 4

# Create placeholders
features_pl = tf.placeholder(tf.float32, [None, feature_size], 'features')
stances_pl = tf.placeholder(tf.int64, [None], 'stances')
keep_prob_pl = tf.placeholder(tf.float32)

# Infer batch size
batch_size = tf.shape(features_pl)[0]

# Define multi-layer perceptron
hidden_layer = tf.nn.dropout(tf.nn.relu(tf.contrib.layers.linear(features_pl, hidden_size)), keep_prob=keep_prob_pl)
logits_flat = tf.nn.dropout(tf.contrib.layers.linear(hidden_layer, target_size), keep_prob=keep_prob_pl)
logits = tf.reshape(logits_flat, [batch_size, target_size])

# Define prediction
softmaxed_logits = tf.nn.softmax(logits)
predict = tf.argmax(softmaxed_logits, 1)

saver = tf.train.Saver()


In [92]:
bow_vectorizer = pickle.load((open("BoW.p", 'rb')))
tfreq_vectorizer = pickle.load((open("Tfreq.p", 'rb')))
tfidf_vectorizer = pickle.load((open("Tfidf.p", 'rb')))

In [93]:
file_test_instances = "Training_Datasets/test_stances_unlabeled.csv"
file_test_bodies = "Training_Datasets/test_bodies.csv"

raw_test = FNCData(file_test_instances, file_test_bodies)
test_set = pipeline_test(raw_test, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer)

In [88]:
test_set[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [77]:
with tf.Session() as sess:
  # Restore variables from disk.
    saver.restore(sess, "./Pretrained_models/mislead.ckpt")
    test_feed_dict = {features_pl: test_set, keep_prob_pl: 1.0}
    test_pred = sess.run(predict, feed_dict=test_feed_dict)


INFO:tensorflow:Restoring parameters from ./Pretrained_models/mislead.ckpt


INFO:tensorflow:Restoring parameters from ./Pretrained_models/mislead.ckpt


In [78]:
test_pred

array([3, 3, 3, ..., 3, 3, 3])

In [79]:
np.unique(test_pred, return_counts = True)

(array([0, 1, 2, 3]), array([ 2772,     8,  4921, 17712]))

In [68]:
len(test_set), len(test_set[0])

(25413, 10001)